In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#file path
file = '../WeatherPy/weather_by_city_output.csv'
weather_df = pd.read_csv(file)
weather_df.head()
weather_df.drop(columns=weather_df.columns[0], axis=1, inplace=True)
weather_df.head()

,city,country,latitude,longitude,max_temp,humidity,cloudiness,windspeed,date
0,Chui,UY,-33.6971,-53.4616,56.16,67,1,8.01,1628879154
1,Hilo,US,19.7297,-155.0900,76.12,83,40,5.75,1628879176
2,Lawrenceburg,US,38.0373,-84.8966,90.72,66,64,10.42,1628879176
3,Kapaa,US,22.0752,-159.3190,80.92,76,40,3.00,1628879176
4,Kirakira,SB,-10.4544,161.9205,77.54,89,83,9.46,1628879177


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure gmaps with api key
gmaps.configure(api_key=g_key)

In [5]:
#lat lng as locations and humidity as weight
locations = weather_df[['latitude', 'longitude']].astype(float)
#convert humidity to float
humidity = weather_df['humidity'].astype(float)

In [6]:
#create humidity heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
#ideal conditions: max temp 65 - 78, humidity <70, windspeed <15
ideal = weather_df.loc[(weather_df['max_temp'] > 65) & (weather_df['max_temp'] <= 78) & 
                       (weather_df['humidity'] < 70) & (weather_df['windspeed'] < 15)]
ideal = ideal.dropna(how='any')
ideal = ideal.reset_index()
ideal = ideal.rename(columns={'index':'weather_df index'})

In [33]:
ideal

,weather_df index,city,country,latitude,longitude,max_temp,humidity,cloudiness,windspeed,date
0,8,Kahama,TZ,-3.8333,32.6000,75.88,40,83,1.86,1628879178
1,20,Atherton,GB,53.5237,-2.4935,69.44,64,28,11.12,1628879084
2,22,Oruzgan,AF,32.8333,66.0000,75.76,12,0,1.57,1628879182
3,75,Karratha,AU,-20.7377,116.8463,65.62,35,0,11.54,1628879195
4,107,Bozeman,US,45.6796,-111.0386,75.69,35,1,4.61,1628878948
5,110,Rawson,AR,-43.3002,-65.1023,65.37,21,0,9.73,1628879204
6,128,Teleneşti,MD,47.4997,28.3656,72.09,51,21,2.01,1628879210
7,149,Preeceville,CA,51.9500,-102.6677,73.00,32,48,12.53,1628879216
8,152,Vodnyy,RU,63.5046,53.4095,74.59,56,58,12.26,1628879217
9,154,Tocopilla,CL,-22.0920,-70.1979,66.04,65,12,10.83,1628879217


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
#store ideal vacation cities in hotels df
hotel_df = ideal
hotel_df

,weather_df index,city,country,latitude,longitude,max_temp,humidity,cloudiness,windspeed,date
0,8,Kahama,TZ,-3.8333,32.6000,75.88,40,83,1.86,1628879178
1,20,Atherton,GB,53.5237,-2.4935,69.44,64,28,11.12,1628879084
2,22,Oruzgan,AF,32.8333,66.0000,75.76,12,0,1.57,1628879182
3,75,Karratha,AU,-20.7377,116.8463,65.62,35,0,11.54,1628879195
4,107,Bozeman,US,45.6796,-111.0386,75.69,35,1,4.61,1628878948
5,110,Rawson,AR,-43.3002,-65.1023,65.37,21,0,9.73,1628879204
6,128,Teleneşti,MD,47.4997,28.3656,72.09,51,21,2.01,1628879210
7,149,Preeceville,CA,51.9500,-102.6677,73.00,32,48,12.53,1628879216
8,152,Vodnyy,RU,63.5046,53.4095,74.59,56,58,12.26,1628879217
9,154,Tocopilla,CL,-22.0920,-70.1979,66.04,65,12,10.83,1628879217


In [39]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

search_term = 'hotel'
radius = 5000
search_type = 'lodging'

hotel_list = []

counter = 0

for city in hotel_df['city']:
    lat = hotel_df['latitude'][counter]
    lng = hotel_df['longitude'][counter]
    location = f'{lat},{lng}'
    counter += 1
    params={'location': location, 'keyword': search_term, 'radius': radius, 'type': search_type, 'key':g_key}
    
    try:
        response = requests.get(base_url, params=params).json()
        hotel_name = response['results'][0]['name']
        hotel_list.append(hotel_name)
        
    except IndexError:
        hotel_list.append(np.nan)

hotel_list

['Submarine Hotels LTD',
 'Holiday Inn Express - Leigh Sports Village',
 nan,
 'Karratha International Hotel',
 'Kimpton Armory Hotel Bozeman',
 'Casa de mar',
 nan,
 nan,
 nan,
 'Hotel Galvarino',
 nan,
 nan,
 'Chibombo District Council Guest House',
 'Pestana Bahia Praia',
 'Hotel Dworek Nad Stawem',
 'Kinghood Joyway Hotel',
 nan,
 "Le Q'Artier",
 'Marinas Club: Restaurante, Pousada e Marina',
 'Hotel Mansión de San José',
 nan,
 'Alexandria56 hotel',
 'Edward Memorial Guest House',
 'Mini-Hotel Lokomotiv',
 'Sprowston Manor Hotel, Golf & Country Club',
 'Sofitel Agadir Thalassa sea & spa',
 'Rosslyn Dimyat Hotel Varna',
 nan,
 nan,
 'POUSADA PONTA DE AREIA - PRADO - BAHIA - HOSPEDAGEM E HOTEL',
 nan,
 "Ha'apai Beach Resort",
 'Tobin Lake Motel',
 nan,
 'Hotel San Miguel',
 'Pousada Encanto Abrolhos',
 nan,
 'Tcham Hotel',
 'Victoria Inn Flin Flon',
 nan,
 nan]

In [42]:
#add hotels tohotels_df and drop any null rows
hotel_df['hotel_name'] = hotel_list
hotel_df = hotel_df.dropna(how='any')
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,weather_df index,city,country,latitude,longitude,max_temp,humidity,cloudiness,windspeed,date,hotel_name
0,8,Kahama,TZ,-3.8333,32.6000,75.88,40,83,1.86,1628879178,Submarine Hotels LTD
1,20,Atherton,GB,53.5237,-2.4935,69.44,64,28,11.12,1628879084,Holiday Inn Express - Leigh Sports Village
2,75,Karratha,AU,-20.7377,116.8463,65.62,35,0,11.54,1628879195,Karratha International Hotel
3,107,Bozeman,US,45.6796,-111.0386,75.69,35,1,4.61,1628878948,Kimpton Armory Hotel Bozeman
4,110,Rawson,AR,-43.3002,-65.1023,65.37,21,0,9.73,1628879204,Casa de mar
5,154,Tocopilla,CL,-22.0920,-70.1979,66.04,65,12,10.83,1628879217,Hotel Galvarino
6,197,Chibombo,ZM,-14.6569,28.0706,67.82,24,0,7.40,1628879229,Chibombo District Council Guest House
7,200,Vila Franca do Campo,PT,37.7167,-25.4333,77.41,69,4,9.73,1628879230,Pestana Bahia Praia
8,207,Ostrów Mazowiecka,PL,52.8024,21.8951,67.62,68,72,7.61,1628879232,Hotel Dworek Nad Stawem
9,210,Baiyin,CN,36.5583,104.2081,65.28,52,78,4.14,1628879232,Kinghood Joyway Hotel


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [46]:
#add marker layer to heat map
marker_coord = hotel_df[['latitude', 'longitude']]
markers = gmaps.marker_layer(marker_coord)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
#output hotel_df to csv
hotel_df.to_csv('vacation_outputs/vacation_ideas.csv')